In [1]:
#Inisialisasi Library

from PIL import Image
from PIL import ImageDraw

import glob
import pandas as pd
from skimage.feature import greycomatrix, greycoprops
from skimage.transform import resize
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import math


Bad key "text.kerning_factor" on line 4 in
C:\Users\ASUS\Anaconda3\envs\tensorflowproject\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
ubah8bit_3bit = [i // 32 for i in range(256)]

In [3]:
# Fungsi-fungsi
# Proses membuka gambar

def read_file(mypath):
    file_name = []
    label = []
    full_path = []

    for path, subdirs, files in os.walk(mypath):
        for name in files:
            full_path.append(os.path.join(path, name))
            label.append(path.split('/')[-1])
            
    return full_path, label

def load_img(img_path):
    hasil_img = Image.open(img_path).convert('L')
    return hasil_img

def load_img_resize(img_path, dim):
    img = Image.open(img_path).convert('L')
    hasil_img = img.resize([dim, dim], Image.NEAREST)
    return hasil_img

def print_img(img):
    plt.imshow(img, cmap='gray')
    
def getLabels(path):
    label = os.listdir(path)
    return label
    
# Preprocessing Gambar
    
def getAllColorIMG(img):
    AllColorIMG = list(img.getdata())
    return AllColorIMG

def ubahKe3bit(AllColorIMG):
    i = 0
    
    for ColorIMG in AllColorIMG:
        AllColorIMG[i] = ubah8bit_3bit[ColorIMG]
        i += 1
        
    return AllColorIMG

def ubahKeMatriks(AllColorIMG):
    matriksIMG = [] 
    i = 0
    
    dimensiGambar = int(math.sqrt(len(AllColorIMG)))
    
    for row in range(dimensiGambar):
        newRowMat = []
        for col in range(dimensiGambar):
            newRowMat.append(AllColorIMG[i])
            i += 1
        matriksIMG.append(newRowMat)
        
    return matriksIMG

def featureExtract(img):
    colorIMG = getAllColorIMG(img)
    _3BitColor = ubahKe3bit(colorIMG)
    matriksIMG = ubahKeMatriks(_3BitColor)
    
    resultOfGLCM = []
    glcm = greycomatrix(matriksIMG, distances=[2], angles=[45], levels=256, symmetric=False, normed=False)
    
    komponen = ['dissimilarity', 'contrast', 'homogeneity', 'energy', 'correlation', 'ASM']
    
    for i in komponen:
        resultOfGLCM.append(greycoprops(glcm, i)[0][0])
    
    return resultOfGLCM

In [4]:
# Memproses File Training dengan mengekstrak fiturnya sebelum dimodelkan tanpa mengubah dimensi gambar

train_path = 'dataset_magang/Training/'
imgType = '*.jpg'

PathOf_trainIMG = []
LabelOf_trainIMG = []
Dissimilarity_train = []
Contrast_train = []
Homogeneity_train = []
Energy_train = []
Correlation_train = []
ASM_train = []

labels = getLabels(train_path)

for label in labels:
    full_paths = glob.glob(train_path + label + '/' + imgType)
    for full_path in full_paths:
        PathOf_trainIMG.append(full_path)
        LabelOf_trainIMG.append(label)
        trainIMG = load_img(full_path)
        featureOfIMG_train = featureExtract(trainIMG)
        Dissimilarity_train.append(featureOfIMG_train[0])
        Contrast_train.append(featureOfIMG_train[1])
        Homogeneity_train.append(featureOfIMG_train[2])
        Energy_train.append(featureOfIMG_train[3])
        Correlation_train.append(featureOfIMG_train[4])
        ASM_train.append(featureOfIMG_train[5])

columns = ["Path", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM" , "Label"]
variables = {"Path":PathOf_trainIMG, "Dissimilarity":Dissimilarity_train, "Contrast":Contrast_train, "Homogeneity":Homogeneity_train, "Energy":Energy_train, "Correlation":Correlation_train, "ASM":ASM_train ,"Label":LabelOf_trainIMG}
df_train = pd.DataFrame(variables, columns = columns)
df_train.to_csv('DataBatik.csv', index=False) 

In [5]:
# Memproses File Validasi dengan mengekstrak fiturnya sebelum dimodelkan

val_path = 'dataset_magang/Validasi/'
imgType = '*.jpg'

PathOf_valIMG = []
LabelOf_valIMG = []
Dissimilarity_val = []
Contrast_val = []
Homogeneity_val = []
Energy_val = []
Correlation_val = []
ASM_val = []

labels = getLabels(val_path)

for label in labels:
    full_paths = glob.glob(val_path + label + '/' + imgType)
    for full_path in full_paths:
        PathOf_valIMG.append(full_path)
        LabelOf_valIMG.append(label)
        valIMG = load_img(full_path)
        featureOfIMG_val = featureExtract(valIMG)
        Dissimilarity_val.append(featureOfIMG_val[0])
        Contrast_val.append(featureOfIMG_val[1])
        Homogeneity_val.append(featureOfIMG_val[2])
        Energy_val.append(featureOfIMG_val[3])
        Correlation_val.append(featureOfIMG_val[4])
        ASM_val.append(featureOfIMG_val[5])

columns = ["Path", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM" , "Label"]
variables = {"Path":PathOf_valIMG, "Dissimilarity":Dissimilarity_val, "Contrast":Contrast_val, "Homogeneity":Homogeneity_val, "Energy":Energy_val, "Correlation":Correlation_val, "ASM":ASM_val ,"Label":LabelOf_valIMG}
df_val = pd.DataFrame(variables, columns = columns)
df_val.to_csv('DataValidasiBatik.csv', index=False) 

In [8]:
# Memproses File Training dengan mengekstrak fiturnya sebelum dimodelkan dengan mengubah dimensi gambar

train_path_rz = 'dataset_magang/Training/'
imgType = '*.jpg'
dim = 224

PathOf_trainIMG_rz = []
LabelOf_trainIMG_rz = []
Dissimilarity_train_rz = []
Contrast_train_rz = []
Homogeneity_train_rz = []
Energy_train_rz = []
Correlation_train_rz = []
ASM_train_rz = []

labels = getLabels(train_path_rz)

for label in labels:
    full_paths = glob.glob(train_path_rz + label + '/' + imgType)
    for full_path in full_paths:
        PathOf_trainIMG_rz.append(full_path)
        LabelOf_trainIMG_rz.append(label)
        trainIMG_rz = load_img_resize(full_path, dim)
        featureOfIMG_train_rz = featureExtract(trainIMG_rz)
        Dissimilarity_train_rz.append(featureOfIMG_train_rz[0])
        Contrast_train_rz.append(featureOfIMG_train_rz[1])
        Homogeneity_train_rz.append(featureOfIMG_train_rz[2])
        Energy_train_rz.append(featureOfIMG_train_rz[3])
        Correlation_train_rz.append(featureOfIMG_train_rz[4])
        ASM_train_rz.append(featureOfIMG_train_rz[5])

columns = ["Path", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM" , "Label"]
variables = {"Path":PathOf_trainIMG_rz, "Dissimilarity":Dissimilarity_train_rz, "Contrast":Contrast_train_rz, "Homogeneity":Homogeneity_train_rz, "Energy":Energy_train_rz, "Correlation":Correlation_train_rz, "ASM":ASM_train_rz ,"Label":LabelOf_trainIMG_rz}
df_train_train_rz = pd.DataFrame(variables, columns = columns)
df_train_train_rz.to_csv('DataBatik_dim224.csv', index=False) 

In [9]:
# Memproses File Validasi dengan mengekstrak fiturnya sebelum dimodelkan dengan mengubah dimensi gambar

val_path = 'dataset_magang/Validasi/'
imgType = '*.jpg'
dim = 224

PathOf_valIMG_rz = []
LabelOf_valIMG_rz = []
Dissimilarity_val_rz = []
Contrast_val_rz = []
Homogeneity_val_rz = []
Energy_val_rz = []
Correlation_val_rz = []
ASM_val_rz = []

labels = getLabels(val_path)

for label in labels:
    full_paths = glob.glob(val_path + label + '/' + imgType)
    for full_path in full_paths:
        PathOf_valIMG_rz.append(full_path)
        LabelOf_valIMG_rz.append(label)
        valIMG_rz = load_img_resize(full_path, dim)
        featureOfIMG_val_rz = featureExtract(valIMG_rz)
        Dissimilarity_val_rz.append(featureOfIMG_val_rz[0])
        Contrast_val_rz.append(featureOfIMG_val_rz[1])
        Homogeneity_val_rz.append(featureOfIMG_val_rz[2])
        Energy_val_rz.append(featureOfIMG_val_rz[3])
        Correlation_val_rz.append(featureOfIMG_val_rz[4])
        ASM_val_rz.append(featureOfIMG_val_rz[5])

columns = ["Path", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM" , "Label"]
variables = {"Path":PathOf_valIMG_rz, "Dissimilarity":Dissimilarity_val_rz, "Contrast":Contrast_val_rz, "Homogeneity":Homogeneity_val_rz, "Energy":Energy_val_rz, "Correlation":Correlation_val_rz, "ASM":ASM_val_rz ,"Label":LabelOf_valIMG_rz}
df_val_rz = pd.DataFrame(variables, columns = columns)
df_val_rz.to_csv('DataValidasiBatik_dim224.csv', index=False) 